# import

In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
tf.__version__

'2.3.1'

# Checkpoint Function

In [13]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & Preprocessing Function

In [14]:
def load_mnist():
    (train_data,train_labels), (test_data,test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data,axis=-1) # [N,28,28] -> [N,28,28,1] : [batch_size,height,width,channel]
    test_data = np.expand_dims(test_data,axis=-1)   # [N,28,28] -> [N,28,28,1] axis=위치(-1=맨끝)
    
    train_data, test_data = normalize(train_data, test_data) # [0~255] -> [0~1]
    
    train_labels = to_categorical(train_labels,10) # [N,] -> [N,10]
    test_labels = to_categorical(test_labels,10) # [N,] -> [N,10] : One hot Encoding
    return train_data, train_labels, test_data, test_labels

def normalize(train_data,test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

# Performance Function

In [15]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels): # 정확도
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits,-1),tf.argmax(labels,-1))
    accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
    return accuracy

def grad(model, images, labels): # gradient
    with tf.GradientTape() as tape:
        loss = loss_fn(model,images,labels)
    return tape.gradient(loss,model.variables)

# Model Function

In [16]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init): # fully connected layer
    return tf.keras.layers.Dense(units=label_dim,use_bias=True,kernel_initializer=weight_init)
# unit = output으로 나갈 Channel이 몇개인가?
# use_bias = bias를 사용하는가 사용하지 않는가?
# kernel_initializer = weight 초기값. 

def sigmoid():
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

# Create model(class version)

In [17]:
class create_model_class(tf.keras.Model):
    def __init__(self,label_dim): # label_dim : Network의 Logit을 구할 때, 최종적으로 몇개의 Output을 낼 것인가?(여기선 10)
        super(create_model_class,self).__init__()
        weight_init = tf.keras.initializers.RandomNormal() # Random Normal Initialization
        
        self.model = tf.keras.Sequential() # 층층이 쌓아나가는 것을 List처럼.
        self.model.add(flatten()) # [N,28,28,1] -> [N,784]
        # [N,784]로 바꿔주는 것은 나중에 fully connected layer를 이용할 것이기 때문. 
        # fully connected가 아니라
        
        for i in range(2):
            self.model.add(dense(256,weight_init))
            self.model.add(sigmoid())
        
        self.model.add(dense(label_dim,weight_init))
        
    def call(self,x,training=None,mask=None):
        x = self.model(x)
        return(x)

# Create model(function version)

In [18]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.RandomNormal()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(2):
        model.add(dense(256,weight_init))
        model.add(sigmoid())
        
    model.add(dense(label_dim,weight_init))
    return model

# Define data & hyper-parameter

In [19]:
"""dataset"""
train_x, train_y, test_x, test_y = load_mnist()

"""parameters"""
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

"""Graph input using Dataset API"""
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = batch_size).\
    batch(batch_size,drop_remainder=True) # batch를 몇개만큼 네트워크에 던져 줄 것인가?
    # shuffle : 데이터셋을 섞는다. input 데이터보다 크게 설정하면 Random으로 shuffle한다.
    # prefetch : batch_size만큼 학습을 하고 있을 때, 미리 메모리에 batch_size만큼 올려놓으면 학습을 좀 더 빠르게 할 수 있다.
    # batch : batch를 몇개만큼 네트워크에 던져 줄 것인가?

test_dataset = tf.data.Dataset.from_tensor_slices((test_x,test_y)).\
    shuffle(buffer_size = 100000).\
    prefetch(buffer_size = len(test_x)).\
    batch(len(test_x))

In [20]:
"""model"""
network = create_model_function(label_dim)

"""training"""
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

"""writer"""
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

# check point의 역할?
# network가 학습을 하다가 끊겼을 때, 다시 재학습을 하고 싶은 경우, 변경되었던 Weight를 불러내는 역할.
# 혹은 Test 이미지에 대해서 정확도를 보고 싶은 경우에도 활용할 수 있음.
# 학습되어 저장된 Weight를 불러오는것.
checkpoint_dir = os.path.join(checkpoint_dir,model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir,model_dir)
logs_dir = os.path.join(logs_dir,model_dir)

# Restore Check point & Start train or test phase

In [21]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exists
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print('[*] Load Success')
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print('[!] Load Failed...')
    
    # train phase
    with summary_writer.as_default(): # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx,(train_input,train_label) in enumerate(train_dataset):
                grads = grad(network,train_input,train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads,network.variables))
                
                train_loss = loss_fn(network,train_input,train_label)
                train_accuracy = accuracy_fn(network,train_input,train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_fn(network,test_input,test_label)
                
                tf.summary.scalar(name='train_loss',data=train_loss,step=counter)
                tf.summary.scalar(name='train_accuracy',data=train_accuracy,step=counter)
                tf.summary.scalar(name='test_accuracy',data=test_accuracy,step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f"\
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))

# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
[!] Load Failed...
Epoch: [ 0] [    0/  468] time: 0.4774, train_loss: 2.34071112, train_accuracy: 0.1406, test_accuracy: 0.1043
Epoch: [ 0] [    1/  468] time: 0.6051, train_loss: 2.29532790, train_accuracy: 0.1094, test_accuracy: 0.1007
Epoch: [ 0] [    2/  468] time: 0.7457, train_loss: 2.29236293, train_accuracy: 0.1875, test_accuracy: 0.1623
Epoch: [ 0] [    3/  468] time: 0.8813, train_loss: 2.26891470, train_accuracy: 0.2422, test_accuracy: 0.2418
Epoch: [ 0] [    4/  468] time: 1.0267, train_loss: 2.27174807, train_accuracy: 0.2266, test_accuracy: 0.2133
Epoch: [ 0] [    5/  468] time: 1.1833, train_loss: 2.31119752, train_accuracy: 0.1875, test_accuracy: 0.1100
Epoch: [ 0] [    6/  468] time: 1.3309, train_loss: 2.25479984, train_accuracy: 0.2344, test_accuracy: 0.1715
Epoch: [ 0] [    7/  468] time: 1.4725, train_loss: 2.27409744, train_accuracy: 0.1875, test_accuracy: 0.1870
Epoch: [ 0] [    8/  468] time: 1.6181, 

Epoch: [ 0] [   75/  468] time: 11.5881, train_loss: 1.03403842, train_accuracy: 0.7031, test_accuracy: 0.7600
Epoch: [ 0] [   76/  468] time: 11.7136, train_loss: 0.98074961, train_accuracy: 0.8203, test_accuracy: 0.7657
Epoch: [ 0] [   77/  468] time: 11.8397, train_loss: 0.99501228, train_accuracy: 0.7656, test_accuracy: 0.7703
Epoch: [ 0] [   78/  468] time: 11.9584, train_loss: 1.02736640, train_accuracy: 0.7812, test_accuracy: 0.7719
Epoch: [ 0] [   79/  468] time: 12.0951, train_loss: 0.95531511, train_accuracy: 0.7656, test_accuracy: 0.7756
Epoch: [ 0] [   80/  468] time: 12.2298, train_loss: 0.97596216, train_accuracy: 0.7734, test_accuracy: 0.7779
Epoch: [ 0] [   81/  468] time: 12.3575, train_loss: 0.85745877, train_accuracy: 0.8047, test_accuracy: 0.7773
Epoch: [ 0] [   82/  468] time: 12.4788, train_loss: 0.96595436, train_accuracy: 0.7891, test_accuracy: 0.7749
Epoch: [ 0] [   83/  468] time: 12.6015, train_loss: 0.92500973, train_accuracy: 0.7578, test_accuracy: 0.7724
E

Epoch: [ 0] [  150/  468] time: 22.1107, train_loss: 0.48156244, train_accuracy: 0.8516, test_accuracy: 0.8749
Epoch: [ 0] [  151/  468] time: 22.2513, train_loss: 0.49452546, train_accuracy: 0.8516, test_accuracy: 0.8756
Epoch: [ 0] [  152/  468] time: 22.3979, train_loss: 0.57070971, train_accuracy: 0.8359, test_accuracy: 0.8774
Epoch: [ 0] [  153/  468] time: 22.5380, train_loss: 0.51659697, train_accuracy: 0.8906, test_accuracy: 0.8789
Epoch: [ 0] [  154/  468] time: 22.6717, train_loss: 0.51567811, train_accuracy: 0.8828, test_accuracy: 0.8806
Epoch: [ 0] [  155/  468] time: 22.7973, train_loss: 0.50517982, train_accuracy: 0.8594, test_accuracy: 0.8802
Epoch: [ 0] [  156/  468] time: 22.9330, train_loss: 0.40798759, train_accuracy: 0.8984, test_accuracy: 0.8771
Epoch: [ 0] [  157/  468] time: 23.0732, train_loss: 0.36350405, train_accuracy: 0.9297, test_accuracy: 0.8751
Epoch: [ 0] [  158/  468] time: 23.2228, train_loss: 0.44978032, train_accuracy: 0.8984, test_accuracy: 0.8739
E

Epoch: [ 0] [  225/  468] time: 33.0399, train_loss: 0.38290620, train_accuracy: 0.8906, test_accuracy: 0.8992
Epoch: [ 0] [  226/  468] time: 33.1845, train_loss: 0.36630195, train_accuracy: 0.9141, test_accuracy: 0.9000
Epoch: [ 0] [  227/  468] time: 33.3566, train_loss: 0.36051545, train_accuracy: 0.9141, test_accuracy: 0.9015
Epoch: [ 0] [  228/  468] time: 33.5082, train_loss: 0.31106812, train_accuracy: 0.9141, test_accuracy: 0.9018
Epoch: [ 0] [  229/  468] time: 33.6757, train_loss: 0.37298477, train_accuracy: 0.9062, test_accuracy: 0.9026
Epoch: [ 0] [  230/  468] time: 33.8452, train_loss: 0.47323972, train_accuracy: 0.8594, test_accuracy: 0.9026
Epoch: [ 0] [  231/  468] time: 34.0187, train_loss: 0.32204434, train_accuracy: 0.9062, test_accuracy: 0.9039
Epoch: [ 0] [  232/  468] time: 34.1713, train_loss: 0.39243591, train_accuracy: 0.8828, test_accuracy: 0.9051
Epoch: [ 0] [  233/  468] time: 34.3489, train_loss: 0.39311489, train_accuracy: 0.8672, test_accuracy: 0.9046
E

Epoch: [ 0] [  299/  468] time: 43.7549, train_loss: 0.33984250, train_accuracy: 0.8984, test_accuracy: 0.9148
Epoch: [ 0] [  300/  468] time: 43.9184, train_loss: 0.37281221, train_accuracy: 0.8750, test_accuracy: 0.9154
Epoch: [ 0] [  301/  468] time: 44.0660, train_loss: 0.24532817, train_accuracy: 0.9375, test_accuracy: 0.9159
Epoch: [ 0] [  302/  468] time: 44.1957, train_loss: 0.23801740, train_accuracy: 0.9297, test_accuracy: 0.9152
Epoch: [ 0] [  303/  468] time: 44.3229, train_loss: 0.25385475, train_accuracy: 0.9219, test_accuracy: 0.9156
Epoch: [ 0] [  304/  468] time: 44.4542, train_loss: 0.26688635, train_accuracy: 0.9219, test_accuracy: 0.9152
Epoch: [ 0] [  305/  468] time: 44.5899, train_loss: 0.26914287, train_accuracy: 0.9219, test_accuracy: 0.9152
Epoch: [ 0] [  306/  468] time: 44.7096, train_loss: 0.32932535, train_accuracy: 0.9062, test_accuracy: 0.9154
Epoch: [ 0] [  307/  468] time: 44.8417, train_loss: 0.29024056, train_accuracy: 0.9219, test_accuracy: 0.9151
E

Epoch: [ 0] [  373/  468] time: 53.3054, train_loss: 0.24614677, train_accuracy: 0.9219, test_accuracy: 0.9214
Epoch: [ 0] [  374/  468] time: 53.4460, train_loss: 0.24950263, train_accuracy: 0.9141, test_accuracy: 0.9222
Epoch: [ 0] [  375/  468] time: 53.5906, train_loss: 0.32313958, train_accuracy: 0.9297, test_accuracy: 0.9221
Epoch: [ 0] [  376/  468] time: 53.7302, train_loss: 0.38311818, train_accuracy: 0.9062, test_accuracy: 0.9220
Epoch: [ 0] [  377/  468] time: 53.8739, train_loss: 0.18195283, train_accuracy: 0.9609, test_accuracy: 0.9220
Epoch: [ 0] [  378/  468] time: 54.0155, train_loss: 0.27992934, train_accuracy: 0.9297, test_accuracy: 0.9199
Epoch: [ 0] [  379/  468] time: 54.1581, train_loss: 0.22816738, train_accuracy: 0.9297, test_accuracy: 0.9201
Epoch: [ 0] [  380/  468] time: 54.2854, train_loss: 0.41030806, train_accuracy: 0.8594, test_accuracy: 0.9204
Epoch: [ 0] [  381/  468] time: 54.4149, train_loss: 0.31569543, train_accuracy: 0.9062, test_accuracy: 0.9207
E

Epoch: [ 0] [  447/  468] time: 62.8710, train_loss: 0.20606598, train_accuracy: 0.9375, test_accuracy: 0.9253
Epoch: [ 0] [  448/  468] time: 62.9937, train_loss: 0.25558114, train_accuracy: 0.9141, test_accuracy: 0.9249
Epoch: [ 0] [  449/  468] time: 63.1174, train_loss: 0.20510536, train_accuracy: 0.9375, test_accuracy: 0.9252
Epoch: [ 0] [  450/  468] time: 63.2408, train_loss: 0.24353299, train_accuracy: 0.9297, test_accuracy: 0.9244
Epoch: [ 0] [  451/  468] time: 63.3626, train_loss: 0.17877153, train_accuracy: 0.9375, test_accuracy: 0.9243
Epoch: [ 0] [  452/  468] time: 63.4863, train_loss: 0.24684313, train_accuracy: 0.9375, test_accuracy: 0.9238
Epoch: [ 0] [  453/  468] time: 63.6119, train_loss: 0.28481400, train_accuracy: 0.8984, test_accuracy: 0.9241
Epoch: [ 0] [  454/  468] time: 63.7352, train_loss: 0.13919403, train_accuracy: 0.9609, test_accuracy: 0.9246
Epoch: [ 0] [  455/  468] time: 63.8588, train_loss: 0.30125204, train_accuracy: 0.9062, test_accuracy: 0.9249
E